<a href="https://colab.research.google.com/github/SunkaraboinaPraveenKumar/Machine_Learning_Projects/blob/main/WeatherForecast_Agent_Pydantic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install pydantic-ai==0.1.3

In [14]:
import nest_asyncio
nest_asyncio.apply()

weather forecast tool

In [15]:
from google.colab import userdata
os.environ["GROQ_API_KEY"]=userdata.get("groq_api_key")
os.environ["WEATHER_API_KEY"]=userdata.get("WEATHER_API_KEY")

In [16]:
import requests
BASE_URL="https://api.openweathermap.org/data/2.5/weather"


def find_weather(city:str)->dict:
  """This function returns the current weather forecast for the given city"""
  units="metric"
  params={
      "q":city,
      "appid":os.getenv('WEATHER_API_KEY'),
      "units":units
  }
  response=requests.get(BASE_URL,params=params)
  if response.status_code==200:
    return response.json()
  else:
    raise Exception(f"Error fetching weather data. Status code: {response.status_code}")

In [17]:
find_weather("Hyderabad")

{'coord': {'lon': 78.4744, 'lat': 17.3753},
 'weather': [{'id': 721,
   'main': 'Haze',
   'description': 'haze',
   'icon': '50d'}],
 'base': 'stations',
 'main': {'temp': 33.23,
  'feels_like': 35.62,
  'temp_min': 32.73,
  'temp_max': 33.23,
  'pressure': 1010,
  'humidity': 46,
  'sea_level': 1010,
  'grnd_level': 948},
 'visibility': 5000,
 'wind': {'speed': 5.14, 'deg': 210},
 'clouds': {'all': 20},
 'dt': 1745380287,
 'sys': {'type': 1,
  'id': 9214,
  'country': 'IN',
  'sunrise': 1745367885,
  'sunset': 1745413427},
 'timezone': 19800,
 'id': 1269843,
 'name': 'Hyderabad',
 'cod': 200}

Pydantic Agent

In [18]:
import os
import requests
from pydantic import BaseModel
from pydantic_ai import Agent, RunContext
from pydantic_ai.settings import ModelSettings

In [19]:
class WeatherForeCast(BaseModel):
  location:str
  description:str
  temperature_celsius:float

In [20]:
weather_agent = Agent(
    model="groq:llama-3.3-70b-versatile",
    model_settings=ModelSettings(temperature=0.2),
    output_type=str,
    system_prompt="You are helpful agent weather assistant. Use the 'get_weather_forecast' tool to find the weather forecast for a given city. Provide clean and friendly answers."
)

In [21]:
@weather_agent.tool
def get_weather_forecast(ctx:RunContext, city:str)->WeatherForeCast:
  """
  Returns current weather forecast for a given city using OpenWeatherMap API.
  """
  url = "https://api.openweathermap.org/data/2.5/weather"
  api_key = os.getenv('WEATHER_API_KEY')
  params = {
      "q": city,
      "appid": api_key,
      "units": "metric"
  }
  response = requests.get(url, params=params).json()
  return WeatherForeCast(
      location=response["name"],
      description=response["weather"][0]["description"].capitalize(),
      temperature_celsius=response["main"]["temp"]
  )

In [24]:
question = input("⛅Ask about the weather:")
result = weather_agent.run_sync(question)
print("\n📍Forecast: ", result.output)

⛅Ask about the weather:Delhi

📍Forecast:  The current weather forecast for Delhi is Haze with a temperature of 31.05 degrees Celsius.
